In [ ]:
import numpy as np
import pandas as pd
import nmrglue as ng
import nibabel as nib

import matplotlib.pyplot as plt

from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.cluster import KMeans

from sklearn.metrics import adjusted_mutual_info_score as AMI

from nmrglue.process.proc_base import fft
from contrastive import CPCA
import imageio
import ants
import pickle

In [ ]:
ms1 = nib.load("/data/bioprotean/Naren2/MRSI/Control04/Control04_REF_complex_FFT_redone.nii")

In [ ]:
ms1_arr = np.array(ms1.dataobj)

In [ ]:
ms1_arr.shape

In [ ]:
RMS01_arr = ms1_arr.reshape(32,64,64,2048,2)

In [ ]:
RMS01_complex_sum = (RMS01_arr[:,:,:,:,0]**2 + RMS01_arr[:,:,:,:,1]**2)**(1/2)

In [ ]:
RMS01_complex_sum.shape

In [ ]:
#saving (samples, features for CPCA)
import nibabel as nib
ni_img = nib.Nifti1Image(RMS01_complex_sum, affine=np.eye(4))
nib.save(ni_img, "/data/bioprotean/Naren2/FURI2/mayomrs_aligned/Control/Control04/Control04_complex_sum_redone.nii")

In [ ]:
postwarp = nib.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS02/RMS02_flair_overlay_norm_Warped.nii.gz').dataobj

In [ ]:
#saving (samples, features for CPCA)
import nibabel as nib
ni_img = nib.Nifti1Image(postwarp_tp, affine=np.eye(4))
nib.save(ni_img, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS02/RMS02_flair_warped.nii")

In [ ]:
RMS01_complex_sum = nib.load("/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS01/RMS01_complex_sum.nii").dataobj
RMS02_complex_sum = nib.load("/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS02/RMS02_complex_sum.nii").dataobj
RMS03_complex_sum = nib.load("/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS03/RMS03_complex_sum.nii").dataobj
RMS05_complex_sum = nib.load("/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS05/RMS05_complex_sum.nii").dataobj
Control01_complex_sum = nib.load("/data/bioprotean/Naren2/FURI2/mayomrs_aligned/Control/Control01/Control01_complex_sum.nii").dataobj
Control02_complex_sum = nib.load("/data/bioprotean/Naren2/FURI2/mayomrs_aligned/Control/Control02/Control02_complex_sum.nii").dataobj
Control03_complex_sum = nib.load("/data/bioprotean/Naren2/FURI2/mayomrs_aligned/Control/Control03/Control03_complex_sum.nii").dataobj
Control04_complex_sum = nib.load("/data/bioprotean/Naren2/FURI2/mayomrs_aligned/Control/Control04/Control04_complex_sum.nii").dataobj

In [ ]:
msl1_arr = np.array(nib.load("/data/bioprotean/Naren2/MRSI/RMS01_V3/RMS01_labels2spectra.nii").dataobj)
msl2_arr = np.array(nib.load("/data/bioprotean/Naren2/MRSI/RMS02_V3/RMS02_labels2spectra.nii").dataobj)
msl3_arr = np.array(nib.load("/data/bioprotean/Naren2/MRSI/RMS03_V3/RMS03_labels2spectra.nii").dataobj)
msl5_arr = np.array(nib.load("/data/bioprotean/Naren2/MRSI/RMS05_V3/RMS05_labels2spectra.nii").dataobj)

In [ ]:
postwarp_tp = np.transpose(postwarp)

In [ ]:
plt.imshow(postwarp_tp[16,:,:], cmap='turbo')
plt.imshow(msl1_arr[16,:,:], alpha=0.5,cmap='Greens')

In [ ]:
#unwarped vs spectral sum
plt.imshow(RMS01_complex_sum[20,:,:,750],cmap='inferno')
plt.imshow(postwarp_tp[20,:,:], alpha=0.7,cmap='Greens')
#plt.imshow(msl1_arr[:,30,:], alpha=0.3,cmap='Greens')

In [ ]:
#making empty lists
ms1_hy_arrays = []
ms1_bgd_arrays = []
ms2_hy_arrays = []
ms2_bgd_arrays = []
ms3_hy_arrays = []
ms3_bgd_arrays = []
ms5_hy_arrays = []
ms5_bgd_arrays = []

In [ ]:
for n in range(0,2048):
    ms1_arr_masked_temp_hy = np.ma.masked_where(msl1_arr<1, RMS01_complex_sum[:,:,:,n]) #mask values that are less than 1 in spectral data (gives hyperintensities)
    hy1_mask = np.ma.getmask(ms1_arr_masked_temp_hy) #get only mask (True False) values
    hy1_mask_applied = np.where(hy1_mask, 0, ms1_arr_masked_temp_hy) #where mask is True (and value is 'masked'), make it 0 and keep everything else
    ms1_hy_arrays.append(hy1_mask_applied)
    ms1_arr_masked_temp_bgd = np.ma.masked_where(msl1_arr>0, RMS01_complex_sum[:,:,:,n]) #mask values that are greater than 0 in spectral data (gives background)
    bgd1_mask = np.ma.getmask(ms1_arr_masked_temp_bgd)
    bg1_mask_applied = np.where(bgd1_mask, 0, ms1_arr_masked_temp_bgd)
    ms1_bgd_arrays.append(bg1_mask_applied)

In [ ]:
#unwarped vs spectral sum
plt.imshow(ms1_bgd_arrays[750][:,30,:],cmap='inferno')
plt.imshow(msl1_arr[:,30,:], alpha=0,cmap='Greens')

In [ ]:
plt.imshow(ms1_hy_arrays[750][:,30,:],cmap='inferno')

In [ ]:
#making mask for different voxel spectra's into one array
ms1_arr_masked_hy = np.stack(ms1_hy_arrays, axis=3)
ms1_arr_masked_bgd = np.stack(ms1_bgd_arrays, axis=3)

In [ ]:
#saving RMS01 files
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/ms1_arr_masked_hy_new.npy',ms1_arr_masked_hy)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/ms1_arr_masked_bgd_new.npy',ms1_arr_masked_bgd)

In [ ]:
#unwarped vs spectral sum
plt.imshow(ms1_arr_masked_bgd[:,30,:,750],cmap='inferno')
plt.imshow(msl1_arr[:,30,:], alpha=0.5,cmap='Greens')

In [ ]:
np.array_equal(ms1_arr_masked_hy,ms1_arr_masked_bgd)

In [ ]:
for n in range(0,2048):
    ms2_arr_masked_temp_hy = np.ma.masked_where(msl2_arr<1, RMS02_complex_sum[:,:,:,n]) #mask values that are less than 1 in spectral data (gives hyperintensities)
    hy2_mask = np.ma.getmask(ms2_arr_masked_temp_hy) #get only mask (True False) values
    hy2_mask_applied = np.where(hy2_mask, 0, ms2_arr_masked_temp_hy) #where mask is True (and value is 'masked'), make it 0 and keep everything else
    ms2_hy_arrays.append(hy2_mask_applied)
    ms2_arr_masked_temp_bgd = np.ma.masked_where(msl2_arr>0, RMS02_complex_sum[:,:,:,n]) #mask values that are greater than 0 in spectral data (gives background)
    bgd2_mask = np.ma.getmask(ms2_arr_masked_temp_bgd)
    bg2_mask_applied = np.where(bgd2_mask, 0, ms2_arr_masked_temp_bgd)
    ms2_bgd_arrays.append(bg2_mask_applied)

In [ ]:
ms2_arr_masked_hy = np.stack(ms2_hy_arrays, axis=3)
ms2_arr_masked_bgd = np.stack(ms2_bgd_arrays, axis=3)

In [ ]:
#saving RMS02 files
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/ms2_arr_masked_hy_new.npy',ms2_arr_masked_hy)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/ms2_arr_masked_bgd_new.npy',ms2_arr_masked_bgd)

In [ ]:
for n in range(0,2048):
    ms3_arr_masked_temp_hy = np.ma.masked_where(msl3_arr<1, RMS03_complex_sum[:,:,:,n]) #mask values that are less than 1 in spectral data (gives hyperintensities)
    hy3_mask = np.ma.getmask(ms3_arr_masked_temp_hy) #get only mask (True False) values
    hy3_mask_applied = np.where(hy3_mask, 0, ms3_arr_masked_temp_hy) #where mask is True (and value is 'masked'), make it 0 and keep everything else
    ms3_hy_arrays.append(hy3_mask_applied)
    ms3_arr_masked_temp_bgd = np.ma.masked_where(msl3_arr>0, RMS03_complex_sum[:,:,:,n]) #mask values that are greater than 0 in spectral data (gives background)
    bgd3_mask = np.ma.getmask(ms3_arr_masked_temp_bgd)
    bg3_mask_applied = np.where(bgd3_mask, 0, ms3_arr_masked_temp_bgd)
    ms3_bgd_arrays.append(bg3_mask_applied)

In [ ]:
ms3_arr_masked_hy = np.stack(ms3_hy_arrays, axis=3)
ms3_arr_masked_bgd = np.stack(ms3_bgd_arrays, axis=3)

In [ ]:
#saving RMS03 files
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS03/redo/ms3_arr_masked_hy_new.npy',ms3_arr_masked_hy)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS03/redo/ms3_arr_masked_bgd_new.npy',ms3_arr_masked_bgd)

In [ ]:
for n in range(0,2048):
    ms5_arr_masked_temp_hy = np.ma.masked_where(msl5_arr<1, RMS05_complex_sum[:,:,:,n]) #mask values that are less than 1 in spectral data (gives hyperintensities)
    hy5_mask = np.ma.getmask(ms5_arr_masked_temp_hy) #get only mask (True False) values
    hy5_mask_applied = np.where(hy5_mask, 0, ms5_arr_masked_temp_hy) #where mask is True (and value is 'masked'), make it 0 and keep everything else
    ms5_hy_arrays.append(hy5_mask_applied)
    ms5_arr_masked_temp_bgd = np.ma.masked_where(msl5_arr>0, RMS05_complex_sum[:,:,:,n]) #mask values that are greater than 0 in spectral data (gives background)
    bgd5_mask = np.ma.getmask(ms5_arr_masked_temp_bgd)
    bg5_mask_applied = np.where(bgd5_mask, 0, ms5_arr_masked_temp_bgd)
    ms5_bgd_arrays.append(bg5_mask_applied)

In [ ]:
ms5_arr_masked_hy = np.stack(ms5_hy_arrays, axis=3)
ms5_arr_masked_bgd = np.stack(ms5_bgd_arrays, axis=3)

In [ ]:
#saving RMS05 files
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/ms5_arr_masked_hy_new.npy',ms5_arr_masked_hy)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/ms5_arr_masked_bgd_new.npy',ms5_arr_masked_bgd)

In [ ]:
#loading control .nii files
Control01_complex_sum = nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/Control/Control01/Control01_complex_sum.nii")
Control02_complex_sum = nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/Control/Control02/Control02_complex_sum.nii")
Control03_complex_sum = nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/Control/Control03/Control03_complex_sum.nii")
Control04_complex_sum = nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/Control/Control04/Control04_complex_sum.nii")

In [ ]:
#converting to array
Control01_complex_sum_arr = np.array(Control01_complex_sum.dataobj)
Control02_complex_sum_arr = np.array(Control02_complex_sum.dataobj)
Control03_complex_sum_arr = np.array(Control03_complex_sum.dataobj)
Control04_complex_sum_arr = np.array(Control04_complex_sum.dataobj)

In [ ]:
#saving control sums as .npy so it's easier to load in
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/Control/Control01/Control01_complex_sum.npy',Control01_complex_sum_arr)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/Control/Control02/Control02_complex_sum.npy',Control02_complex_sum_arr)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/Control/Control03/Control03_complex_sum.npy',Control03_complex_sum_arr)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/Control/Control04/Control04_complex_sum.npy',Control04_complex_sum_arr)

In [ ]:
#loading all necessary files in one cell                                                                                                                    ###############
ms1_arr_masked_hy = np.load('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/ms1_arr_masked_hy_new.npy')
ms2_arr_masked_hy = np.load('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/ms2_arr_masked_hy_new.npy')
ms3_arr_masked_hy = np.load('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS03/redo/ms3_arr_masked_hy_new.npy')
ms5_arr_masked_hy = np.load('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/ms5_arr_masked_hy_new.npy')
ms1_arr_masked_bgd = np.load('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/ms1_arr_masked_bgd_new.npy')
ms2_arr_masked_bgd = np.load('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/ms2_arr_masked_bgd_new.npy')
ms3_arr_masked_bgd = np.load('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS03/redo/ms3_arr_masked_bgd_new.npy')
ms5_arr_masked_bgd = np.load('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/ms5_arr_masked_bgd_new.npy')

In [ ]:
#loading in saved files
Control01_complex_sum = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/Control/Control01/Control01_complex_sum.npy')
Control02_complex_sum = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/Control/Control02/Control02_complex_sum.npy')
Control03_complex_sum = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/Control/Control03/Control03_complex_sum.npy')
Control04_complex_sum = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/Control/Control04/Control04_complex_sum.npy')

In [ ]:
#reshaping before concatenating
ms1_arr_masked_hy_re = ms1_arr_masked_hy.reshape(32*64*64,2048)
ms2_arr_masked_hy_re = ms2_arr_masked_hy.reshape(32*64*64,2048)
ms3_arr_masked_hy_re = ms3_arr_masked_hy.reshape(32*64*64,2048)
ms5_arr_masked_hy_re = ms5_arr_masked_hy.reshape(32*64*64,2048)
ms1_arr_masked_bgd_re = ms1_arr_masked_bgd.reshape(32*64*64,2048)
ms2_arr_masked_bgd_re = ms2_arr_masked_bgd.reshape(32*64*64,2048)
ms3_arr_masked_bgd_re = ms3_arr_masked_bgd.reshape(32*64*64,2048)
ms5_arr_masked_bgd_re = ms5_arr_masked_bgd.reshape(32*64*64,2048)

In [ ]:
#saving masked arrays which are reshaped as .npy so it's easier to load in
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/ms1_arr_masked_hy_re.npy',ms1_arr_masked_hy_re)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/ms2_arr_masked_hy_re.npy',ms2_arr_masked_hy_re)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS03/redo/ms3_arr_masked_hy_re.npy',ms3_arr_masked_hy_re)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/ms5_arr_masked_hy_re.npy',ms5_arr_masked_hy_re)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/ms1_arr_masked_bgd_re.npy',ms1_arr_masked_bgd_re)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/ms2_arr_masked_bgd_re.npy',ms2_arr_masked_bgd_re)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS03/redo/ms3_arr_masked_bgd_re.npy',ms3_arr_masked_bgd_re)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/ms5_arr_masked_bgd_re.npy',ms5_arr_masked_bgd_re)

In [ ]:
Control01_complex_sum_re = Control01_complex_sum.reshape(32*64*64,2048)
Control02_complex_sum_re = Control02_complex_sum.reshape(32*64*64,2048)
Control03_complex_sum_re = Control03_complex_sum.reshape(32*64*64,2048)
Control04_complex_sum_re = Control04_complex_sum.reshape(32*64*64,2048)

In [ ]:
#saving control sums reshaped as .npy so it's easier to load in
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/Control/Control01/Control01_complex_sum_re.npy',Control01_complex_sum_re)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/Control/Control02/Control02_complex_sum_re.npy',Control02_complex_sum_re)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/Control/Control03/Control03_complex_sum_re.npy',Control03_complex_sum_re)
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/Control/Control04/Control04_complex_sum_re.npy',Control04_complex_sum_re)

In [ ]:
#loading all reshaped files in one cell
ms1_arr_masked_hy_re = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/ms1_arr_masked_hy_re.npy')
ms2_arr_masked_hy_re = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/ms2_arr_masked_hy_re.npy')
ms3_arr_masked_hy_re = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS03/redo/ms3_arr_masked_hy_re.npy')
ms5_arr_masked_hy_re = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/ms5_arr_masked_hy_re.npy')
ms1_arr_masked_bgd_re = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/ms1_arr_masked_bgd_re.npy')
ms2_arr_masked_bgd_re = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/ms2_arr_masked_bgd_re.npy')
ms3_arr_masked_bgd_re = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS03/redo/ms3_arr_masked_bgd_re.npy')
ms5_arr_masked_bgd_re = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/ms5_arr_masked_bgd_re.npy')
Control01_complex_sum_re = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/Control/Control01/Control01_complex_sum_re.npy')
Control02_complex_sum_re = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/Control/Control02/Control02_complex_sum_re.npy')
Control03_complex_sum_re = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/Control/Control03/Control03_complex_sum_re.npy')
Control04_complex_sum_re = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/Control/Control04/Control04_complex_sum_re.npy')

In [ ]:
np.count_nonzero(ms1_arr_masked_hy_re==0)

In [ ]:
np.count_nonzero(ms1_arr_masked_hy_re)/2048

In [ ]:
np.count_nonzero(ms5_arr_masked_hy_re)/2048

In [ ]:
#bgd_frame = np.concatenate((Control01_complex_sum_re,Control02_complex_sum_re,Control03_complex_sum_re,Control04_complex_sum_re, ms1_arr_masked_bgd_re, ms2_arr_masked_bgd_re, ms3_arr_masked_bgd_re, ms5_arr_masked_bgd_re), axis=0)
hy_frame = np.concatenate((ms1_arr_masked_hy_re, ms2_arr_masked_hy_re, ms3_arr_masked_hy_re, ms5_arr_masked_hy_re), axis=0)
#, ms3_arr_masked_hy_re, ms5_arr_masked_hy_re

In [ ]:
hy_test = np.concatenate((ms1_arr_masked_hy_re, ms2_arr_masked_hy_re, ms3_arr_masked_hy_re, ms5_arr_masked_hy_re), axis=0)

In [ ]:
bgd_frame_onlyc = np.concatenate((Control01_complex_sum_re,Control02_complex_sum_re,Control03_complex_sum_re,Control04_complex_sum_re), axis=0)
#hy_frame_allms = np.concatenate((ms1_arr_masked_hy_re, ms2_arr_masked_hy_re, ms3_arr_masked_hy_re, ms5_arr_masked_hy_re, ms1_arr_masked_bgd_re, ms2_arr_masked_bgd_re, ms3_arr_masked_bgd_re, ms5_arr_masked_bgd_re), axis=0)
#, ms3_arr_masked_hy_re, ms5_arr_masked_hy_re
#, ms3_arr_masked_bgd_re, ms5_arr_masked_bgd_re

In [ ]:
#ms everything outside hy
bgd_onlyms = np.concatenate((ms1_arr_masked_bgd_re, ms2_arr_masked_bgd_re, ms3_arr_masked_bgd_re, ms5_arr_masked_bgd_re), axis=0)

In [ ]:
#print(bgd_frame.shape)
print(bgd_frame_onlyc.shape)
print(hy_frame.shape)
#print(hy_frame_allms.shape)

In [ ]:
#print(bgd_frame.real.shape[1]==hy_frame.real.shape[1])
#print(bgd_frame_onlyc.real.shape[1]==hy_frame.real.shape[1])
print(bgd_frame_onlyc.real.shape[1]==hy_frame_allms.real.shape[1])

In [ ]:
print(bgd_frame_onlyc.real.shape[1]==hy_frame.real.shape[1])

In [ ]:
#making memory for CPCA
del ms1_arr_masked_hy_re
del ms2_arr_masked_hy_re
#del ms3_arr_masked_hy_re
#del ms5_arr_masked_hy_re
del ms1_arr_masked_bgd_re
del ms2_arr_masked_bgd_re
#del ms3_arr_masked_bgd_re
#del ms5_arr_masked_bgd_re
del Control01_complex_sum_re
del Control02_complex_sum_re
del Control03_complex_sum_re
del Control04_complex_sum_re
#del bgd_frame_onlyc
#del hy_frame_allms

In [ ]:
#cpca = CPCA()
#labels = [0]*131072+[1]*131072+[2]*131072+[3]*131072
#cpca_transformed = cpca.fit_transform(hy_frame.real, bgd_frame.real, preprocess_with_pca_dim=500, plot=True, active_labels=labels, n_alphas=10)

In [ ]:
hy_frame_allms.shape

In [ ]:
cpca = CPCA()
labels = [0]*131072+[1]*131072+[2]*131072+[3]*131072+[4]*131072+[5]*131072+[6]*131072+[7]*131072
cpca_transformed = cpca.fit_transform(hy_frame_allms.real, bgd_frame_onlyc.real, plot=True, active_labels=labels, n_alphas=10)
#preprocess_with_pca_dim=500

In [ ]:
hy_frame.shape

In [ ]:
cpca = CPCA()
labels = [0]*131072+[1]*131072+[2]*131072+[3]*131072
cpca_transformed = cpca.fit_transform(hy_frame.real, bgd_frame_onlyc.real, plot=True, active_labels=labels, n_alphas=10)
#preprocess_with_pca_dim=500

In [ ]:
#updated cpca with new labels
cpca = CPCA()
labels = [0]*131072+[1]*131072+[2]*131072+[3]*131072
cpca_transformed = cpca.fit_transform(hy_frame.real, bgd_frame_onlyc.real, plot=True, active_labels=labels, n_alphas=10,)
#preprocess_with_pca_dim=500

In [ ]:
#updated cpca with new labels
cpca = CPCA(standardize=False)
labels = [0]*131072+[1]*131072+[2]*131072+[3]*131072
cpca_transformed = cpca.fit_transform(hy_frame.real, bgd_frame_onlyc.real, plot=True, active_labels=labels, n_alphas=10,)
#preprocess_with_pca_dim=500

In [ ]:
x = plt.scatter(principalComponents[:,1], principalComponents[:,0], c = colors)
classes = (['RMS01', 'RMS02', 'RMS03', 'RMS05'])
plt.legend(handles=x.legend_elements()[0], labels=classes)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

principalComponents = pca.fit_transform(hy_frame.real)

In [ ]:
colors = [0]*131072+[1]*131072+[2]*131072+[3]*131072

In [ ]:
cpca = CPCA()
labels = [0]*131072+[1]*131072
cpca_transformed = cpca.fit_transform(hy_frame.real, bgd_frame_onlyc.real, active_labels=labels, alpha_selection='manual', alpha_value=3, n_alphas_to_return=1)
#preprocess_with_pca_dim=500

In [ ]:
cpca = CPCA()
labels = [0]*131072+[1]*131072+[2]*131072+[3]*131072
cpca_transformed2 = cpca.fit_transform(hy_frame.real, bgd_frame_onlyc.real, active_labels=labels, alpha_selection='manual', alpha_value=2, n_alphas_to_return=1)
#preprocess_with_pca_dim=500

In [ ]:
cpca = CPCA()
labels = [0]*131072+[1]*131072+[2]*131072+[3]*131072
cpca_transformed3 = cpca.fit_transform(hy_frame.real, bgd_frame_onlyc.real, active_labels=labels, alpha_selection='manual', alpha_value=3, n_alphas_to_return=1)
#preprocess_with_pca_dim=500

In [ ]:
cpca = CPCA()
labels = [0]*131072+[1]*131072+[2]*131072+[3]*131072
cpca_transformed4 = cpca.fit_transform(hy_frame.real, bgd_frame_onlyc.real, active_labels=labels, alpha_selection='manual', alpha_value=1)
#preprocess_with_pca_dim=500

In [ ]:
cpca = CPCA()
labels = [0]*131072+[1]*131072+[2]*131072+[3]*131072
cpca_transformed5 = cpca.fit_transform(hy_frame.real, bgd_frame_onlyc.real, active_labels=labels, alpha_selection='manual', alpha_value=5, n_alphas_to_return=1)
#preprocess_with_pca_dim=500

In [ ]:
cpca = CPCA()
labels = [0]*131072+[1]*131072+[2]*131072+[3]*131072
cpca_transformed6 = cpca.fit_transform(hy_frame.real, bgd_frame_onlyc.real, active_labels=labels, alpha_selection='manual', alpha_value=0, n_alphas_to_return=1)
#preprocess_with_pca_dim=500

In [ ]:
cpca_transformed[1][:,0].shape

In [ ]:
plt.scatter(cpca_transformed[1][:,0], cpca_transformed[1][:,1])

In [ ]:
plt.scatter(cpca_transformed3[:,0], cpca_transformed3[:,1])

In [ ]:
#cpca with redone labels with alpha = 3
plt.scatter(cpca_transformed3[:,0], cpca_transformed3[:,1])

In [ ]:
#cpca with redone labels with alpha = 1
plt.scatter(cpca_transformed4[:,0], cpca_transformed4[:,1])

In [ ]:
#cpca with redone labels with alpha = 5
plt.scatter(cpca_transformed5[:,0], cpca_transformed5[:,1])

In [ ]:
from sklearn.cluster import KMeans

data = list(zip(cpca_transformed3[:,0], cpca_transformed3[:,1]))
inertias = []

for i in range(1,5):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(data)
    inertias.append(kmeans.inertia_)

plt.plot(range(1,5), inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
from sklearn.cluster import KMeans

data = list(zip(cpca_transformed3[:,0], cpca_transformed3[:,1]))
inertias = []

for i in range(1,6):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(data)
    inertias.append(kmeans.inertia_)

plt.plot(range(1,6), inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(data)

plt.scatter(cpca_transformed3[:,0], cpca_transformed3[:,1], c=kmeans.labels_)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(data)

plt.scatter(cpca_transformed3[:,0], cpca_transformed3[:,1], c=kmeans.labels_)
plt.show()

In [ ]:
from sklearn.cluster import KMeans

data = list(zip(cpca_transformed4[:,0], cpca_transformed4[:,1]))
inertias = []

for i in range(1,5):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(data)
    inertias.append(kmeans.inertia_)

plt.plot(range(1,5), inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3)
data_clusters = kmeans.fit_predict(data)

LABEL_COLOR_MAP = {0:'red', 1 : 'green' ,2 : 'blue'}
label_color=[LABEL_COLOR_MAP[l] for l in data_clusters]

plt.scatter(cpca_transformed4[:,0], cpca_transformed4[:,1], c=label_color)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4)
data_clusters = kmeans.fit_predict(data)

LABEL_COLOR_MAP = {0:'black', 1 : 'cyan' ,2 : 'orange', 3: 'red'}
label_color=[LABEL_COLOR_MAP[l] for l in data_clusters]

plt.scatter(cpca_transformed4[:,0], cpca_transformed4[:,1], c=label_color)
plt.show()

In [ ]:
(kmeans.labels_ == data_clusters).all()

In [ ]:
labels = np.reshape(data_clusters, (524288,1))

In [ ]:
unique_values, counts = np.unique(labels, return_counts=True)

print(unique_values) 
print(counts)

In [ ]:
hy_frame_wlabel = np.append(hy_frame, labels, axis = 1)

In [ ]:
ms1_arr_masked_hy = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/ms1_arr_masked_hy_new.npy')

In [ ]:
ms2_arr_masked_hy = np.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/ms2_arr_masked_hy_new.npy')

In [ ]:
ms1_arr_hy_wlabels_re = ms1_arr_masked_hy_re.reshape(32,64,64,2048)

In [ ]:
ms1_arr_hy_wlabels = hy_frame_wlabel[0:131072]
ms2_arr_hy_wlabels = hy_frame_wlabel[131072:262144]
ms3_arr_hy_wlabels = hy_frame_wlabel[262144:393216]
ms5_arr_hy_wlabels = hy_frame_wlabel[393216:524288]

In [ ]:
ms1_arr_hy_wlabels.shape

In [ ]:
ms1_arr_hy_wlabels_re = ms1_arr_hy_wlabels.reshape(32,64,64,2049)
ms2_arr_hy_wlabels_re = ms2_arr_hy_wlabels.reshape(32,64,64,2049)
ms3_arr_hy_wlabels_re = ms3_arr_hy_wlabels.reshape(32,64,64,2049)
ms5_arr_hy_wlabels_re = ms5_arr_hy_wlabels.reshape(32,64,64,2049)

In [ ]:
ms1_arr_hy_wlabels_re.shape

In [ ]:
plt.imshow(ms1_arr_masked_hy[14,:,:,750],cmap='inferno')

In [ ]:
plt.imshow(ms1_arr_hy_wlabels_re[14,:,:,2048],cmap='inferno')

In [ ]:
ms1_arr_hy_wlabels_re_onlylabels = ms1_arr_hy_wlabels_re[:,:,:,2048]
ms2_arr_hy_wlabels_re_onlylabels = ms2_arr_hy_wlabels_re[:,:,:,2048]
ms3_arr_hy_wlabels_re_onlylabels = ms3_arr_hy_wlabels_re[:,:,:,2048]
ms5_arr_hy_wlabels_re_onlylabels = ms5_arr_hy_wlabels_re[:,:,:,2048]

In [ ]:
np.unique(ms1_arr_hy_wlabels_re_onlylabels)

In [ ]:
np.unique(ms2_arr_hy_wlabels_re_onlylabels)

In [ ]:
np.unique(ms3_arr_hy_wlabels_re_onlylabels)

In [ ]:
np.unique(ms5_arr_hy_wlabels_re_onlylabels)

In [ ]:
#saving (samples, features for CPCA)
import nibabel as nib
ni_img = nib.Nifti1Image(ms5_arr_hy_wlabels_re_onlylabels, affine=np.eye(4))
nib.save(ni_img, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/ms5_arr_hy_wlabels_re_onlylabels_4c.nii")

In [ ]:
#saving (samples, features for CPCA)
import nibabel as nib
ni_img = nib.Nifti1Image(ms1_arr_masked_hy[:,:,:,1000], affine=np.eye(4))
nib.save(ni_img, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/ms1_arr_masked_hy_new.nii")

In [ ]:
ms1_arr_hy_wlabels_re_onlylabels = nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/ms1_arr_hy_wlabels_re_onlylabels_4c.nii").dataobj
ms2_arr_hy_wlabels_re_onlylabels = nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/ms2_arr_hy_wlabels_re_onlylabels_4c.nii").dataobj
ms3_arr_hy_wlabels_re_onlylabels = nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS03/redo/ms3_arr_hy_wlabels_re_onlylabels_4c.nii").dataobj
ms5_arr_hy_wlabels_re_onlylabels = nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/ms5_arr_hy_wlabels_re_onlylabels_4c.nii").dataobj

In [ ]:
flair_test_ss_2 = np.array(nib.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/RMS02_flair_ss_256.nii').dataobj)

In [ ]:
Control1_spectra = nib.load("/data/MRSI/Naren2/MRSI/Control01/Control01_spectra_one.nii").dataobj
Control2_spectra = nib.load("/data/MRSI/Naren2/MRSI/Control02/Control02_spectra_one.nii").dataobj
Control3_spectra = nib.load("/data/MRSI/Naren2/MRSI/Control03/Control03_spectra_one.nii").dataobj
Control4_spectra = nib.load("/data/MRSI/Naren2/MRSI/Control04/Control04_spectra_one.nii").dataobj

RMS01_spectra = nib.load("/data/MRSI/Naren2/MRSI/RMS01_V3/RMS01_spectra_one.nii").dataobj
RMS02_spectra = nib.load("/data/MRSI/Naren2/MRSI/RMS02_V3/RMS02_spectra_one.nii").dataobj
RMS03_spectra = nib.load("/data/MRSI/Naren2/MRSI/RMS03_V3/RMS03_spectra_one.nii").dataobj
RMS05_spectra = nib.load("/data/MRSI/Naren2/MRSI/RMS05_V3/RMS05_spectra_one.nii").dataobj

Control1_spectra_ss_labels = imageio.imread("/data/MRSI/Naren2/MRSI/Control01/Control01_spectra_one_ss_labels.tiff").transpose()
Control2_spectra_ss_labels = imageio.imread("/data/MRSI/Naren2/MRSI/Control02/Control02_spectra_one_ss_labels.tif").transpose()
Control3_spectra_ss_labels = imageio.imread("/data/MRSI/Naren2/MRSI/Control03/Control03_spectra_one_ss_labels.tif").transpose()
Control4_spectra_ss_labels = imageio.imread("/data/MRSI/Naren2/MRSI/Control04/Control04_spectra_one_ss_labels.tiff").transpose()

RMS01_spectra_ss_labels = imageio.imread("/data/MRSI/Naren2/MRSI/RMS01_V3/RMS01_spectra_one_ss_labels.tiff").transpose()
RMS02_spectra_ss_labels = imageio.imread("/data/MRSI/Naren2/MRSI/RMS02_V3/RMS02_spectra_one_ss_labels.tiff").transpose()
RMS03_spectra_ss_labels = imageio.imread("/data/MRSI/Naren2/MRSI/RMS03_V3/RMS03_spectra_one_ss_labels.tiff").transpose()
RMS05_spectra_ss_labels = imageio.imread("/data/MRSI/Naren2/MRSI/RMS05_V3/RMS05_spectra_one_ss_labels.tif").transpose()

In [ ]:
RMS01_ss = np.ma.masked_where(RMS01_spectra_ss_labels<1, RMS01_spectra) #mask values that are less than 1 in spectral data (gives hyperintensities)
RMS01_ss_mask = np.ma.getmask(RMS01_ss) #get only mask (True False) values
RMS01_ss = np.where(RMS01_ss_mask, 0, RMS01_ss) #where mask is True (and value is 'masked'), make it 0 and keep everything else
RMS02_ss = np.ma.masked_where(RMS02_spectra_ss_labels<1, RMS02_spectra) #mask values that are less than 1 in spectral data (gives hyperintensities)
RMS03_ss = np.ma.masked_where(RMS03_spectra_ss_labels<1, RMS03_spectra) #mask values that are less than 1 in spectral data (gives hyperintensities)
RMS05_ss = np.ma.masked_where(RMS05_spectra_ss_labels<1, RMS05_spectra) #mask values that are less than 1 in spectral data (gives hyperintensities)

In [ ]:
RMS01_ss = np.ma.masked_where(RMS01_spectra_ss_labels<1, RMS01_spectra) #mask values that are less than 1 in spectral data (gives hyperintensities)
RMS01_ss_mask = np.ma.getmask(RMS01_ss) #get only mask (True False) values
RMS01_ss = np.where(RMS01_ss_mask, 0, RMS01_ss) #where mask is True (and value is 'masked'), make it 0 and keep everything else

RMS02_ss = np.ma.masked_where(RMS02_spectra_ss_labels<1, RMS02_spectra) #mask values that are less than 1 in spectral data (gives hyperintensities)
RMS02_ss_mask = np.ma.getmask(RMS02_ss) #get only mask (True False) values
RMS02_ss = np.where(RMS02_ss_mask, 0, RMS02_ss) #where mask is True (and value is 'masked'), make it 0 and keep everything else

RMS03_ss = np.ma.masked_where(RMS03_spectra_ss_labels<1, RMS03_spectra) #mask values that are less than 1 in spectral data (gives hyperintensities)
RMS03_ss_mask = np.ma.getmask(RMS03_ss) #get only mask (True False) values
RMS03_ss = np.where(RMS03_ss_mask, 0, RMS03_ss) #where mask is True (and value is 'masked'), make it 0 and keep everything else

RMS05_ss = np.ma.masked_where(RMS05_spectra_ss_labels<1, RMS05_spectra) #mask values that are less than 1 in spectral data (gives hyperintensities)
RMS05_ss_mask = np.ma.getmask(RMS05_ss) #get only mask (True False) values
RMS05_ss = np.where(RMS05_ss_mask, 0, RMS05_ss) #where mask is True (and value is 'masked'), make it 0 and keep everything else

In [ ]:
flair_1 = np.array(nib.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/ms/RMS01/RMS01_flair_ss_256.nii').dataobj)
flair_2 = np.array(nib.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/RMS02_flair_ss_256.nii').dataobj)
flair_3 = np.array(nib.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/ms/RMS03/RMS03_flair_ss_256.nii').dataobj)
flair_5 = np.array(nib.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/RMS05_flair_ss_256.nii').dataobj)

In [ ]:
flair_test_ss_2_tp = np.transpose(flair_test_ss_2, (1,0,2))

In [ ]:
flair_1_tp = np.rot90(np.fliplr(flair_1))
flair_2_tp = np.rot90(np.fliplr(flair_2))
flair_3_tp = np.flip(np.transpose(flair_3, (2,1,0)))
flair_5_tp = np.rot90(np.fliplr(flair_5))

In [ ]:
plt.imshow(flair_5_tp[140,:,:],cmap='inferno')

In [ ]:
plt.imshow(RMS05_ss[23,:,:],cmap='inferno')

In [ ]:
RMS01_spectra_ants = ants.from_numpy(RMS01_ss)
RMS02_spectra_ants = ants.from_numpy(RMS02_ss)
RMS03_spectra_ants = ants.from_numpy(RMS03_ss)
RMS05_spectra_ants = ants.from_numpy(RMS05_ss)

RMS01_flair_ants = ants.from_numpy(flair_1_tp)
RMS02_flair_ants = ants.from_numpy(flair_2_tp)
RMS03_flair_ants = ants.from_numpy(flair_3_tp)
RMS05_flair_ants = ants.from_numpy(flair_5_tp)

In [ ]:
RMS01_flair_ants_re = ants.resample_image(RMS01_flair_ants, (32, 64, 64), use_voxels=True, interp_type=1)
RMS02_flair_ants_re = ants.resample_image(RMS02_flair_ants, (32, 64, 64), use_voxels=True, interp_type=1)
RMS03_flair_ants_re = ants.resample_image(RMS03_flair_ants, (32, 64, 64), use_voxels=True, interp_type=1)
RMS05_flair_ants_re = ants.resample_image(RMS05_flair_ants, (32, 64, 64), use_voxels=True, interp_type=1)

In [ ]:
RMS01_flair_ants_re.set_spacing((1,1,1))
RMS02_flair_ants_re.set_spacing((1,1,1))
RMS03_flair_ants_re.set_spacing((1,1,1))
RMS05_flair_ants_re.set_spacing((1,1,1))

In [ ]:
ants.plot(RMS05_flair_ants_re, overlay=RMS05_spectra_ants, overlay_alpha = 0.5, axis=0, nslices=3, figsize=4)

In [ ]:
RMS01_flair2spectra = ants.registration(fixed = RMS01_spectra_ants,
                                moving = RMS01_flair_ants_re, 
                                type_of_transform = 'SyN',aff_metric='mattes')

In [ ]:
RMS02_flair2spectra = ants.registration(fixed = RMS02_spectra_ants,
                                moving = RMS02_flair_ants_re, 
                                type_of_transform = 'SyN',aff_metric='mattes')

In [ ]:
RMS03_flair2spectra = ants.registration(fixed = RMS03_spectra_ants,
                                moving = RMS03_flair_ants_re, 
                                type_of_transform = 'SyN',aff_metric='mattes')

In [ ]:
RMS05_flair2spectra = ants.registration(fixed = RMS05_spectra_ants,
                                moving = RMS05_flair_ants_re, 
                                type_of_transform = 'SyN',aff_metric='mattes')

In [ ]:
import pickle 
  
try: 
    geeky_file = open('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/RMS05_flair2spectradict', 'wb') 
    pickle.dump(RMS05_flair2spectra, geeky_file) 
    geeky_file.close() 
  
except: 
    print("Something went wrong")

In [ ]:
with open('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/RMS01_flair2spectradict', 'rb') as f:
    RMS01_flair2spectra = pickle.load(f)
with open('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/RMS02_flair2spectradict', 'rb') as f:
    RMS02_flair2spectra = pickle.load(f)
with open('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS03/redo/RMS03_flair2spectradict', 'rb') as f:
    RMS03_flair2spectra = pickle.load(f)
with open('/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/RMS05_flair2spectradict', 'rb') as f:
    RMS05_flair2spectra = pickle.load(f)

In [ ]:
RMS05_flair2spectra

In [ ]:
ants.plot(RMS02_flair_ants_re, overlay=RMS02_spectra_ants, overlay_alpha = 0.6, axis=2, nslices=3, figsize=4)

In [ ]:
ants.plot(RMS02_flair2spectra['warpedmovout'], overlay=RMS02_spectra_ants, overlay_alpha = 0.6, axis=2, nslices=3, figsize=4)

In [ ]:
RMS01_flair2spectra['warpedmovout']

In [ ]:
#warping flair2spectra directly to highres
RMS01_flair2spectra_re_highres = ants.resample_image(RMS01_flair2spectra['warpedmovout'], (256, 256, 256), use_voxels=True, interp_type=1)
RMS02_flair2spectra_re_highres = ants.resample_image(RMS02_flair2spectra['warpedmovout'], (256, 256, 256), use_voxels=True, interp_type=1)
RMS03_flair2spectra_re_highres = ants.resample_image(RMS03_flair2spectra['warpedmovout'], (256, 256, 256), use_voxels=True, interp_type=1)
RMS05_flair2spectra_re_highres = ants.resample_image(RMS05_flair2spectra['warpedmovout'], (256, 256, 256), use_voxels=True, interp_type=1)
RMS01_flair2spectra_re_highres.set_spacing((1,1,1))
RMS02_flair2spectra_re_highres.set_spacing((1,1,1))
RMS03_flair2spectra_re_highres.set_spacing((1,1,1))
RMS05_flair2spectra_re_highres.set_spacing((1,1,1))

In [ ]:
RMS01_flair2spectra_re_highres

In [ ]:
RMS01_flair2spectra_2highres = ants.registration(fixed = RMS01_flair_ants,
                                moving = RMS01_flair2spectra_re_highres, 
                                type_of_transform = 'SyN',aff_metric='mattes')
RMS02_flair2spectra_2highres = ants.registration(fixed = RMS02_flair_ants,
                                moving = RMS02_flair2spectra_re_highres, 
                                type_of_transform = 'SyN',aff_metric='mattes')
RMS03_flair2spectra_2highres = ants.registration(fixed = RMS03_flair_ants,
                                moving = RMS03_flair2spectra_re_highres, 
                                type_of_transform = 'SyN',aff_metric='mattes')
RMS05_flair2spectra_2highres = ants.registration(fixed = RMS05_flair_ants,
                                moving = RMS05_flair2spectra_re_highres, 
                                type_of_transform = 'SyN',aff_metric='mattes')

In [ ]:
ms1_arr_hy_wlabels_re_onlylabels = np.array(nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/ms1_arr_hy_wlabels_re_onlylabels_4c.nii").dataobj)
ms2_arr_hy_wlabels_re_onlylabels = np.array(nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/ms2_arr_hy_wlabels_re_onlylabels_4c.nii").dataobj)
ms3_arr_hy_wlabels_re_onlylabels = np.array(nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS03/redo/ms3_arr_hy_wlabels_re_onlylabels_4c.nii").dataobj)
ms5_arr_hy_wlabels_re_onlylabels = np.array(nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/ms5_arr_hy_wlabels_re_onlylabels_4c.nii").dataobj)

In [ ]:
RMS01_hy_clustered_ants = ants.from_numpy(ms1_arr_hy_wlabels_re_onlylabels)
RMS02_hy_clustered_ants = ants.from_numpy(ms2_arr_hy_wlabels_re_onlylabels)
RMS03_hy_clustered_ants = ants.from_numpy(ms3_arr_hy_wlabels_re_onlylabels)
RMS05_hy_clustered_ants = ants.from_numpy(ms5_arr_hy_wlabels_re_onlylabels)

In [ ]:
#warping lowres clustered labels directly to highres using fwdtransforms
RMS01_hy_clustered_ants_re_highres = ants.resample_image(RMS01_hy_clustered_ants, (256, 256, 256), use_voxels=True, interp_type=1)
RMS02_hy_clustered_ants_re_highres = ants.resample_image(RMS02_hy_clustered_ants, (256, 256, 256), use_voxels=True, interp_type=1)
RMS03_hy_clustered_ants_re_highres = ants.resample_image(RMS03_hy_clustered_ants, (256, 256, 256), use_voxels=True, interp_type=1)
RMS05_hy_clustered_ants_re_highres = ants.resample_image(RMS05_hy_clustered_ants, (256, 256, 256), use_voxels=True, interp_type=1)
RMS01_hy_clustered_ants_re_highres.set_spacing((1,1,1))
RMS02_hy_clustered_ants_re_highres.set_spacing((1,1,1))
RMS03_hy_clustered_ants_re_highres.set_spacing((1,1,1))
RMS05_hy_clustered_ants_re_highres.set_spacing((1,1,1))

In [ ]:
hylabelsRMS01_clustered_2highres = ants.apply_transforms(fixed=RMS01_flair_ants, moving=RMS01_hy_clustered_ants_re_highres, transformlist= RMS01_flair2spectra_2highres['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)
hylabelsRMS02_clustered_2highres = ants.apply_transforms(fixed=RMS02_flair_ants, moving=RMS02_hy_clustered_ants_re_highres, transformlist= RMS02_flair2spectra_2highres['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)
hylabelsRMS03_clustered_2highres = ants.apply_transforms(fixed=RMS03_flair_ants, moving=RMS03_hy_clustered_ants_re_highres, transformlist= RMS03_flair2spectra_2highres['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)
hylabelsRMS05_clustered_2highres = ants.apply_transforms(fixed=RMS05_flair_ants, moving=RMS05_hy_clustered_ants_re_highres, transformlist= RMS05_flair2spectra_2highres['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)

In [ ]:
ants.plot(RMS02_flair_ants_re, overlay=RMS02_hy_clustered_ants, overlay_alpha = 0.6, axis=1, nslices=5, figsize=4)

In [ ]:
ants.plot(RMS02_flair_ants, overlay=hylabelsRMS02_clustered_2highres, overlay_alpha = 0.6, axis=1, nslices=5, figsize=4)

In [ ]:
hylabelsRMS01_clustered_2highres_resampled_npy = hylabelsRMS01_clustered_2highres.numpy()
hylabelsRMS02_clustered_2highres_resampled_npy = hylabelsRMS02_clustered_2highres.numpy()
hylabelsRMS03_clustered_2highres_resampled_npy = hylabelsRMS03_clustered_2highres.numpy()
hylabelsRMS05_clustered_2highres_resampled_npy = hylabelsRMS05_clustered_2highres.numpy()

In [ ]:
hylabelsRMS01_clustered_2highres.unique()

In [ ]:
#saving hyperintensity label clusters projected back to high res as numpy
ni_img = nib.Nifti1Image(hylabelsRMS01_clustered_2highres_resampled_npy, affine=np.eye(4))
nib.save(ni_img, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/hylabelsRMS01_clustered_2highres_resampled.nii")
ni_img2 = nib.Nifti1Image(hylabelsRMS02_clustered_2highres_resampled_npy, affine=np.eye(4))
nib.save(ni_img2, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/hylabelsRMS02_clustered_2highres_resampled.nii")
ni_img3 = nib.Nifti1Image(hylabelsRMS03_clustered_2highres_resampled_npy, affine=np.eye(4))
nib.save(ni_img3, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS03/redo/hylabelsRMS03_clustered_2highres_resampled.nii")
ni_img4 = nib.Nifti1Image(hylabelsRMS05_clustered_2highres_resampled_npy, affine=np.eye(4))
nib.save(ni_img4, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/hylabelsRMS05_clustered_2highres_resampled.nii")

In [ ]:
atlas = np.array(nib.load('/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/icbm452_atlas_warp5_sinc.nii').dataobj)

In [ ]:
atlas.shape

In [ ]:
#removing last dimension
atlas = atlas[:,:,:,0]

In [ ]:
atlas_tp = np.transpose(atlas)
atlas_tp_vflip = np.fliplr(atlas_tp)
atlas_tp_hflip = np.flipud(atlas_tp_vflip)

In [ ]:
atlas_tp_hflip.shape

In [ ]:
plt.imshow(atlas_tp_hflip[:,100,:],cmap='inferno')

In [ ]:
plt.imshow(RMS02_flair_ants.numpy()[:,130,:], cmap='inferno')

In [ ]:
atlas_ants = ants.from_numpy(atlas_tp_hflip)

In [ ]:
#IMPORTANT STEP. Make sure atlas has float datatype. Otherwise, registration won't work well
atlas_ants = atlas_ants.clone(pixeltype = 'float')

In [ ]:
#warping flair2spectra directly to atlas
RMS01_flair2spectra_re_atlas = ants.resample_image(RMS01_flair2spectra['warpedmovout'], (148, 188, 149), use_voxels=True, interp_type=1)
RMS02_flair2spectra_re_atlas = ants.resample_image(RMS02_flair2spectra['warpedmovout'], (148, 188, 149), use_voxels=True, interp_type=1)
RMS03_flair2spectra_re_atlas = ants.resample_image(RMS03_flair2spectra['warpedmovout'], (148, 188, 149), use_voxels=True, interp_type=1)
RMS05_flair2spectra_re_atlas = ants.resample_image(RMS05_flair2spectra['warpedmovout'], (148, 188, 149), use_voxels=True, interp_type=1)
RMS01_flair2spectra_re_atlas.set_spacing((1,1,1))
RMS02_flair2spectra_re_atlas.set_spacing((1,1,1))
RMS03_flair2spectra_re_atlas.set_spacing((1,1,1))
RMS05_flair2spectra_re_atlas.set_spacing((1,1,1))

In [ ]:
atlas_ants

In [ ]:
RMS01_flair2spectra_re_atlas

In [ ]:
RMS01_flair2spectra_2atlas = ants.registration(fixed = atlas_ants,
                                moving = RMS01_flair2spectra_re_atlas, 
                                type_of_transform = 'SyN',aff_metric='mattes')
RMS02_flair2spectra_2atlas = ants.registration(fixed = atlas_ants,
                                moving = RMS02_flair2spectra_re_atlas, 
                                type_of_transform = 'SyN',aff_metric='mattes')
RMS03_flair2spectra_2atlas = ants.registration(fixed = atlas_ants,
                                moving = RMS03_flair2spectra_re_atlas, 
                                type_of_transform = 'SyN',aff_metric='mattes')
RMS05_flair2spectra_2atlas = ants.registration(fixed = atlas_ants,
                                moving = RMS05_flair2spectra_re_atlas, 
                                type_of_transform = 'SyN',aff_metric='mattes')

In [ ]:
RMS01_flair2spectra_2atlas[]

In [ ]:
#warping lowres clustered labels directly to atlas using fwdtransforms
RMS01_hy_clustered_ants_re_atlas = ants.resample_image(RMS01_hy_clustered_ants, (148, 188, 149), use_voxels=True, interp_type=1)
RMS02_hy_clustered_ants_re_atlas = ants.resample_image(RMS02_hy_clustered_ants, (148, 188, 149), use_voxels=True, interp_type=1)
RMS03_hy_clustered_ants_re_atlas = ants.resample_image(RMS03_hy_clustered_ants, (148, 188, 149), use_voxels=True, interp_type=1)
RMS05_hy_clustered_ants_re_atlas = ants.resample_image(RMS05_hy_clustered_ants, (148, 188, 149), use_voxels=True, interp_type=1)
RMS01_hy_clustered_ants_re_atlas.set_spacing((1,1,1))
RMS02_hy_clustered_ants_re_atlas.set_spacing((1,1,1))
RMS03_hy_clustered_ants_re_atlas.set_spacing((1,1,1))
RMS05_hy_clustered_ants_re_atlas.set_spacing((1,1,1))

In [ ]:
hylabelsRMS01_clustered_2atlas = ants.apply_transforms(fixed=atlas_ants, moving=RMS01_hy_clustered_ants_re_atlas, transformlist= RMS01_flair2spectra_2atlas['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)
hylabelsRMS02_clustered_2atlas = ants.apply_transforms(fixed=atlas_ants, moving=RMS02_hy_clustered_ants_re_atlas, transformlist= RMS02_flair2spectra_2atlas['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)
hylabelsRMS03_clustered_2atlas = ants.apply_transforms(fixed=atlas_ants, moving=RMS03_hy_clustered_ants_re_atlas, transformlist= RMS03_flair2spectra_2atlas['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)
hylabelsRMS05_clustered_2atlas = ants.apply_transforms(fixed=atlas_ants, moving=RMS05_hy_clustered_ants_re_atlas, transformlist= RMS05_flair2spectra_2atlas['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)

In [ ]:
ants.plot(atlas_ants, overlay=hylabelsRMS02_clustered_2atlas, overlay_alpha = 0.6, axis=1, nslices=5, figsize=4)

In [ ]:
hylabelsRMS01_clustered_2atlas_resampled_npy = hylabelsRMS01_clustered_2atlas.numpy()
hylabelsRMS02_clustered_2atlas_resampled_npy = hylabelsRMS02_clustered_2atlas.numpy()
hylabelsRMS03_clustered_2atlas_resampled_npy = hylabelsRMS03_clustered_2atlas.numpy()
hylabelsRMS05_clustered_2atlas_resampled_npy = hylabelsRMS05_clustered_2atlas.numpy()

In [ ]:
np.unique(hylabelsRMS05_clustered_2atlas_resampled_npy)

In [ ]:
#saving hyperintensity label clusters projected back to high res as numpy
ni_img = nib.Nifti1Image(hylabelsRMS01_clustered_2atlas_resampled_npy, affine=np.eye(4))
nib.save(ni_img, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/hylabelsRMS01_clustered_2atlas.nii")
ni_img2 = nib.Nifti1Image(hylabelsRMS02_clustered_2atlas_resampled_npy, affine=np.eye(4))
nib.save(ni_img2, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/hylabelsRMS02_clustered_2atlas.nii")
ni_img3 = nib.Nifti1Image(hylabelsRMS03_clustered_2atlas_resampled_npy, affine=np.eye(4))
nib.save(ni_img3, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS03/redo/hylabelsRMS03_clustered_2atlas.nii")
ni_img4 = nib.Nifti1Image(hylabelsRMS05_clustered_2atlas_resampled_npy, affine=np.eye(4))
nib.save(ni_img4, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/hylabelsRMS05_clustered_2atlas.nii")

In [ ]:
test1 = nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/hylabelsRMS01_clustered_2atlas.nii").dataobj
test2 = nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/hylabelsRMS02_clustered_2atlas.nii").dataobj
test3 = nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS03/redo/hylabelsRMS03_clustered_2atlas.nii").dataobj
test5 = nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/hylabelsRMS05_clustered_2atlas.nii").dataobj

In [ ]:
np.unique(test1)

In [ ]:
np.unique(test2)

In [ ]:
np.unique(test3)

In [ ]:
np.unique(test5)

In [ ]:
RMS01_flair_ants_re = ants.resample_image(RMS01_flair_ants, (148, 188, 149), use_voxels=True, interp_type=1)
RMS02_flair_ants_re = ants.resample_image(RMS02_flair_ants, (148, 188, 149), use_voxels=True, interp_type=1)
RMS03_flair_ants_re = ants.resample_image(RMS03_flair_ants, (148, 188, 149), use_voxels=True, interp_type=1)
RMS05_flair_ants_re = ants.resample_image(RMS05_flair_ants, (148, 188, 149), use_voxels=True, interp_type=1)

In [ ]:
RMS01_flair_ants_re

In [ ]:
ants.plot(RMS01_flair_ants_re, overlay=atlas_ants, overlay_alpha = 0.5, axis=0, nslices=15, figsize=4)

In [ ]:
RMS01_flair2atlas = ants.registration(fixed = atlas_ants, initial_transform = None,
                                moving = RMS01_flair_ants_re, 
                                type_of_transform = 'SyN',aff_metric='mattes')

In [ ]:
RMS02_flair2atlas = ants.registration(fixed = atlas_ants,
                                moving = RMS02_flair_ants_re, 
                                type_of_transform = 'SyN',aff_metric='mattes')

In [ ]:
RMS03_flair2atlas = ants.registration(fixed = atlas_ants,
                                moving = RMS03_flair_ants_re, 
                                type_of_transform = 'SyN',aff_metric='mattes')

In [ ]:
RMS05_flair2atlas = ants.registration(fixed = atlas_ants,
                                moving = RMS05_flair_ants_re, 
                                type_of_transform = 'SyN',aff_metric='mattes')

In [ ]:
import pickle 
  
try: 
    geeky_file = open('/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS01_flair2atlas', 'wb') 
    pickle.dump(RMS01_flair2atlas, geeky_file)
    geeky_file.close() 
  
except: 
    print("Something went wrong")

In [ ]:
with open('/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS01_flair2atlas', 'rb') as f:
    RMS01_flair2atlas = pickle.load(f)
with open('/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS02_flair2atlas', 'rb') as f:
    RMS02_flair2atlas = pickle.load(f)
with open('/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS03_flair2atlas', 'rb') as f:
    RMS03_flair2atlas = pickle.load(f)
with open('/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS05_flair2atlas', 'rb') as f:
    RMS05_flair2atlas = pickle.load(f)

In [ ]:
ants.plot(RMS01_flair2atlas['warpedmovout'], overlay=atlas_ants, overlay_alpha = 0.3, axis=2, nslices=15, figsize=4)

In [ ]:
hylabelsRMS01_clustered_2highres = np.array(nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS01/redo/hylabelsRMS01_clustered_2highres_resampled.nii").dataobj)
hylabelsRMS02_clustered_2highres = np.array(nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS02/redo/hylabelsRMS02_clustered_2highres_resampled.nii").dataobj)
hylabelsRMS03_clustered_2highres = np.array(nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS03/redo/hylabelsRMS03_clustered_2highres_resampled.nii").dataobj)
hylabelsRMS05_clustered_2highres = np.array(nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/ms/RMS05/redo/hylabelsRMS05_clustered_2highres_resampled.nii").dataobj)

In [ ]:
hylabelsRMS01_clustered_2highres_ants = ants.from_numpy(hylabelsRMS01_clustered_2highres)
hylabelsRMS02_clustered_2highres_ants = ants.from_numpy(hylabelsRMS02_clustered_2highres)
hylabelsRMS03_clustered_2highres_ants = ants.from_numpy(hylabelsRMS03_clustered_2highres)
hylabelsRMS05_clustered_2highres_ants = ants.from_numpy(hylabelsRMS05_clustered_2highres)

In [ ]:
hylabelsRMS01_clustered_2highres_ants

In [ ]:
hylabelsRMS01_clustered_2highres_ants_re = ants.resample_image(hylabelsRMS01_clustered_2highres_ants, (148, 188, 149), use_voxels=True, interp_type=1)
hylabelsRMS02_clustered_2highres_ants_re = ants.resample_image(hylabelsRMS02_clustered_2highres_ants, (148, 188, 149), use_voxels=True, interp_type=1)
hylabelsRMS03_clustered_2highres_ants_re = ants.resample_image(hylabelsRMS03_clustered_2highres_ants, (148, 188, 149), use_voxels=True, interp_type=1)
hylabelsRMS05_clustered_2highres_ants_re = ants.resample_image(hylabelsRMS05_clustered_2highres_ants, (148, 188, 149), use_voxels=True, interp_type=1)

In [ ]:
hylabelsRMS01_clustered_2highres_ants_re.set_spacing((1,1,1))
hylabelsRMS02_clustered_2highres_ants_re.set_spacing((1,1,1))
hylabelsRMS03_clustered_2highres_ants_re.set_spacing((1,1,1))
hylabelsRMS05_clustered_2highres_ants_re.set_spacing((1,1,1))

In [ ]:
RMS01_labels2atlas = ants.apply_transforms(fixed=atlas_ants, moving=hylabelsRMS01_clustered_2highres_ants_re, transformlist= RMS01_flair2atlas['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)
RMS02_labels2atlas = ants.apply_transforms(fixed=atlas_ants, moving=hylabelsRMS02_clustered_2highres_ants_re, transformlist= RMS02_flair2atlas['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)
RMS03_labels2atlas = ants.apply_transforms(fixed=atlas_ants, moving=hylabelsRMS03_clustered_2highres_ants_re, transformlist= RMS03_flair2atlas['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)
RMS05_labels2atlas = ants.apply_transforms(fixed=atlas_ants, moving=hylabelsRMS05_clustered_2highres_ants_re, transformlist= RMS05_flair2atlas['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)

In [ ]:
ants.plot(RMS02_flair2atlas['warpedmovout'], overlay=RMS02_labels2atlas, overlay_alpha = 0.6, axis=2, nslices=5, figsize=4)

In [ ]:
ants.plot(RMS01_labels2atlas, overlay=atlas_ants, overlay_alpha = 0.5, axis=2, nslices=5, figsize=4)

In [ ]:
hylabelsRMS01_highreslabels = np.array(nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/ms/RMS01/RMS01_Label_ss_256.nii").dataobj)
hylabelsRMS02_highreslabels = np.array(nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/ms/RMS02/RMS02_Label_ss_256.nii").dataobj)
hylabelsRMS03_highreslabels = np.array(nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/ms/RMS03/RMS03_Label_fixed.nii").dataobj)
hylabelsRMS05_highreslabels = np.array(nib.load("/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/ms/RMS05/RMS05_Label_fixed.nii").dataobj)

In [ ]:
hylabelsRMS05_highreslabels.shape

In [ ]:
hylabelsRMS01_highreslabels_tp = np.rot90(np.fliplr(hylabelsRMS01_highreslabels))
hylabelsRMS02_highreslabels_tp = np.rot90(np.fliplr(hylabelsRMS02_highreslabels))
hylabelsRMS03_highreslabels_tp = np.flipud(np.transpose(hylabelsRMS03_highreslabels))
hylabelsRMS05_highreslabels_tp = np.transpose(np.flipud(np.fliplr(np.transpose(np.flip(hylabelsRMS05_highreslabels)))), (1, 0, 2))

In [ ]:
#make labels match flair
labels_iso_5_flip_tp = np.transpose(labels_iso_5_flip)
labels_iso_5_flip_tp_lr = np.fliplr(labels_iso_5_flip_tp)
labels_iso_5_flip_tp_lr_ud = np.flipud(labels_iso_5_flip_tp_lr)
labels_iso_5_flip_tp_lr_ud_tp = np.transpose(labels_iso_5_flip_tp_lr_ud, (1, 0, 2))

In [ ]:
plt.imshow(flair_5_tp[90,:,:], cmap='inferno')

In [ ]:
n=90
plt.imshow(flair_5_tp[n,:,:], cmap='inferno')
plt.imshow(hylabelsRMS05_highreslabels_tp[n,:,:], cmap='inferno', alpha=0.3)

In [ ]:
hylabelsRMS01_highreslabels_tp_ants = ants.from_numpy(hylabelsRMS01_highreslabels_tp)
hylabelsRMS02_highreslabels_tp_ants = ants.from_numpy(hylabelsRMS02_highreslabels_tp)
hylabelsRMS03_highreslabels_tp_ants = ants.from_numpy(hylabelsRMS03_highreslabels_tp)
hylabelsRMS05_highreslabels_tp_ants = ants.from_numpy(hylabelsRMS05_highreslabels_tp)

In [ ]:
ants.plot(hylabelsRMS01_highreslabels_tp_ants, overlay=RMS01_flair_ants, overlay_alpha = 0.5, axis=2, nslices=5, figsize=4)

In [ ]:
ants.plot(hylabelsRMS05_highreslabels_tp_ants, overlay=RMS05_flair_ants, overlay_alpha = 0.5, axis=2, nslices=5, figsize=4)

In [ ]:
hylabelsRMS01_highreslabels_tp_ants_re = ants.resample_image(hylabelsRMS01_highreslabels_tp_ants, (148, 188, 149), use_voxels=True, interp_type=1)
hylabelsRMS02_highreslabels_tp_ants_re = ants.resample_image(hylabelsRMS02_highreslabels_tp_ants, (148, 188, 149), use_voxels=True, interp_type=1)
hylabelsRMS03_highreslabels_tp_ants_re = ants.resample_image(hylabelsRMS03_highreslabels_tp_ants, (148, 188, 149), use_voxels=True, interp_type=1)
hylabelsRMS05_highreslabels_tp_ants_re = ants.resample_image(hylabelsRMS05_highreslabels_tp_ants, (148, 188, 149), use_voxels=True, interp_type=1)

In [ ]:
hylabelsRMS01_highreslabels_tp_ants_re.set_spacing((1,1,1))
hylabelsRMS02_highreslabels_tp_ants_re.set_spacing((1,1,1))
hylabelsRMS03_highreslabels_tp_ants_re.set_spacing((1,1,1))
hylabelsRMS05_highreslabels_tp_ants_re.set_spacing((1,1,1))

In [ ]:
RMS01_highreslabels2atlas = ants.apply_transforms(fixed=atlas_ants, moving=hylabelsRMS01_highreslabels_tp_ants_re, transformlist= RMS01_flair2atlas['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)
RMS02_highreslabels2atlas = ants.apply_transforms(fixed=atlas_ants, moving=hylabelsRMS02_highreslabels_tp_ants_re, transformlist= RMS02_flair2atlas['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)
RMS03_highreslabels2atlas = ants.apply_transforms(fixed=atlas_ants, moving=hylabelsRMS03_highreslabels_tp_ants_re, transformlist= RMS03_flair2atlas['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)
RMS05_highreslabels2atlas = ants.apply_transforms(fixed=atlas_ants, moving=hylabelsRMS05_highreslabels_tp_ants_re, transformlist= RMS05_flair2atlas['fwdtransforms'], interpolator='nearestNeighbor', verbose=True)

In [ ]:
ants.plot(RMS02_flair2atlas['warpedmovout'], overlay=RMS02_highreslabels2atlas, overlay_alpha = 0.5, axis=2, nslices=5, figsize=4)

In [ ]:
ants.plot(atlas_ants, overlay=RMS02_highreslabels2atlas, overlay_alpha = 0.5, axis=2, nslices=5, figsize=4)

In [ ]:
RMS01_labels2atlas_npy = RMS01_labels2atlas.numpy()
RMS02_labels2atlas_npy = RMS02_labels2atlas.numpy()
RMS03_labels2atlas_npy = RMS03_labels2atlas.numpy()
RMS05_labels2atlas_npy = RMS05_labels2atlas.numpy()
atlas_ants_npy = atlas_ants.numpy()
RMS01_flair2atlas_npy = RMS01_flair2atlas['warpedmovout'].numpy()
RMS02_flair2atlas_npy = RMS02_flair2atlas['warpedmovout'].numpy()
RMS03_flair2atlas_npy = RMS03_flair2atlas['warpedmovout'].numpy()
RMS05_flair2atlas_npy = RMS05_flair2atlas['warpedmovout'].numpy()
RMS01_highreslabels2atlas_npy = RMS01_highreslabels2atlas.numpy()
RMS02_highreslabels2atlas_npy = RMS02_highreslabels2atlas.numpy()
RMS03_highreslabels2atlas_npy = RMS03_highreslabels2atlas.numpy()
RMS05_highreslabels2atlas_npy = RMS05_highreslabels2atlas.numpy()

In [ ]:
RMS01_flair_ants_re_npy = RMS01_flair_ants_re.numpy()

In [ ]:
ni_img = nib.Nifti1Image(RMS01_labels2atlas_npy, affine=np.eye(4))
nib.save(ni_img, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS01_labels2atlas.nii")
ni_img2 = nib.Nifti1Image(RMS02_labels2atlas_npy, affine=np.eye(4))
nib.save(ni_img2, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS02_labels2atlas.nii")
ni_img3 = nib.Nifti1Image(RMS03_labels2atlas_npy, affine=np.eye(4))
nib.save(ni_img3, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS03_labels2atlas.nii")
ni_img4 = nib.Nifti1Image(RMS05_labels2atlas_npy, affine=np.eye(4))
nib.save(ni_img4, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS05_labels2atlas.nii")
ni_img5 = nib.Nifti1Image(atlas_ants_npy, affine=np.eye(4))
nib.save(ni_img5, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/atlas_ants.nii")

In [ ]:
ni_img6 = nib.Nifti1Image(RMS01_flair2atlas_npy, affine=np.eye(4))
nib.save(ni_img6, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS01_flair2atlas.nii")
ni_img7 = nib.Nifti1Image(RMS02_flair2atlas_npy, affine=np.eye(4))
nib.save(ni_img7, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS02_flair2atlas.nii")
ni_img8 = nib.Nifti1Image(RMS03_flair2atlas_npy, affine=np.eye(4))
nib.save(ni_img8, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS03_flair2atlas.nii")
ni_img9 = nib.Nifti1Image(RMS05_flair2atlas_npy, affine=np.eye(4))
nib.save(ni_img9, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS05_flair2atlas.nii")

In [ ]:
ni_img10 = nib.Nifti1Image(RMS01_highreslabels2atlas_npy, affine=np.eye(4))
nib.save(ni_img10, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS01_highreslabels2atlas.nii")
ni_img11 = nib.Nifti1Image(RMS02_highreslabels2atlas_npy, affine=np.eye(4))
nib.save(ni_img11, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS02_highreslabels2atlas.nii")
ni_img12 = nib.Nifti1Image(RMS03_highreslabels2atlas_npy, affine=np.eye(4))
nib.save(ni_img12, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS03_highreslabels2atlas.nii")
ni_img13 = nib.Nifti1Image(RMS05_highreslabels2atlas_npy, affine=np.eye(4))
nib.save(ni_img13, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS05_highreslabels2atlas.nii")
ni_img14 = nib.Nifti1Image(atlas_ants_npy, affine=np.eye(4))
nib.save(ni_img14, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/atlas_ants.nii")

In [ ]:
ni_img10 = nib.Nifti1Image(RMS01_flair_ants_re_npy, affine=np.eye(4))
nib.save(ni_img10, "/data/MRSI/Naren2/FURI2/mayomrs_aligned/NarenTemplateSpace/Atlas_registration/RMS01_flair_ants_re.nii")

In [ ]:
hylabelsRMS02_clustered_2highres_resampled

In [ ]:
ants.plot(RMS02_flair_ants, overlay=hylabelsRMS02_clustered_2highres_resampled, overlay_alpha = 1, axis=1, nslices=10, figsize=4)

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(data)

plt.scatter(cpca_transformed3[:,0], cpca_transformed3[:,1], c=kmeans.labels_)
plt.show()

In [ ]:
np.save('/data/bioprotean/Naren2/FURI2/mayomrs_aligned/CPCA/RMS01_RMS02_CPCA_Alpha3.npy',cpca_transformed3)

In [ ]:
plt.scatter(cpca_transformed2[:,0], cpca_transformed2[:,1])

In [ ]:
cpca_transformed[0]

In [ ]:
#attempting to separate all hyperintensities from control background, black is hyperintensities, each color is diff pt.
cpca2 = CPCA()
labels2 = [0]*524288+[1]*131072+[2]*131072+[3]*131072+[4]*131072
cpca_transformed2 = cpca.fit_transform(hy_frame_allms.real, bgd_frame_onlyc.real, plot=True, active_labels=labels2, n_alphas=10)
#preprocess_with_pca_dim=500

In [ ]:
#attempting to separate all hyperintensities from control background, red is control, black is hyperintensities
cpca2 = CPCA()
labels2 = [0]*524288+[1]*524288
cpca_transformed2 = cpca2.fit_transform(hy_frame_allms.real, bgd_frame_onlyc.real, plot=True, active_labels=labels2, n_alphas=10, preprocess_with_pca_dim=500)

In [ ]:
#hy_frame is only hyperintensity spectra to see how each brain separates from one another
cpca2 = CPCA()
labels2 = [0]*131072+[1]*131072+[2]*131072+[3]*131072
cpca_transformed2 = cpca2.fit_transform(hy_frame.real, bgd_frame_onlyc.real, plot=True, active_labels=labels2, n_alphas=10, preprocess_with_pca_dim=500)

In [ ]:
#hy_test is only 1st and 2nd MS brains, with first 250k being hyperintensties spectra, and second 250k being control background spectra
#hy_test was used to reduce data size
cpca2 = CPCA()
labels2 = [0]*262144+[1]*262144
cpca_transformed2 = cpca2.fit_transform(hy_test.real, bgd_frame_onlyc.real, plot=True, active_labels=labels2, n_alphas=10, preprocess_with_pca_dim=500)

In [ ]:
#without standardizing
cpca2 = CPCA(standardize=False, verbose=True)
labels2 = [0]*262144+[1]*262144
cpca_transformed2 = cpca2.fit_transform(hy_test.real, bgd_frame_onlyc.real, plot=True, active_labels=labels2, n_alphas=10, preprocess_with_pca_dim=500)

In [ ]:
cpca2 = CPCA(verbose=True)
labels2 = [1]*262144+[0]*262144
cpca_transformed2 = cpca2.fit_transform(hy_test.real, bgd_frame_onlyc.real, plot=True, active_labels=labels2, n_alphas=10, preprocess_with_pca_dim=500)

In [ ]:
np.count_nonzero(hy_test==0)

In [ ]:
#only ms with bgd outside ms hy
cpca2 = CPCA(verbose=True)
labels2 = [0]*131072+[1]*131072+[2]*131072+[3]*131072
cpca_transformed2 = cpca2.fit_transform(hy_test.real, bgd_onlyms.real, plot=True, active_labels=labels2, n_alphas=10, preprocess_with_pca_dim=500)